In [54]:
import fitz
import pandas as pd

In [58]:
doc = fitz.open("../misc/1-4.pdf")

for page in doc:
    text = page.get_text("words", sort=True)
    print(page.mediabox_size[1])

<class 'float'>


In [34]:
table_coordinates = "1-4.pdf,1,251,535,573,326"

table_x0 = 251.0
table_y0 = 535.0

table_x1 = 573.0
table_y1 = 326.0

print(type(text))

words_positions = []

for item in text:
    
    item_x0 = item[0]
    item_y0 = 824.0 - item[1]

    item_x1 = item[2]
    item_y1 = 824.0 - item[3]

    # print(item_x0)
    # print(item_y0)
    # print(item_x1)
    # print(item_y1)
    # print()

    if item_x0 > table_x0:
        if item_y0 < table_y0:
            if item_x1 < table_x1:
                if item_y1 > table_y1:
                    
                    avg_y = (item_y0 + item_y1) / 2.0
                    text = item[4]
                    block_no = item[5]

                    word_pos = [avg_y, text, block_no]

                    words_positions.append(word_pos)


<class 'list'>


In [40]:
raw_lines = []

raw_line = []
prev_pos = words_positions[0][0]

for word in words_positions:
    
    curr_pos = word[0]

    # If decided that it is the same line
    if abs(curr_pos - prev_pos) < 5:
        raw_line.append(word)
    
    # If decided that it is not the same line
    else:

        # Commit the line
        raw_lines.append(raw_line)

        # Clear the previous line
        raw_line = []

        # Append the word for new line
        raw_line.append(word)

    prev_pos = curr_pos


print(raw_lines)
        

[[[519.1793975830078, 'EFF', 24], [519.1793975830078, 'Code(%)', 24], [519.1793975830078, 'Pmpp(W)', 39], [519.1793975830078, 'Vmpp(V)', 54], [519.1793975830078, 'Impp(A)', 69], [519.1793975830078, 'Voc(V)', 84], [519.1793975830078, 'Isc(A)', 99]], [[504.4794616699219, '22.6%', 20], [503.767578125, '5.69', 35], [503.67578125, '0.579', 50], [503.67578125, '9.835', 65], [503.67578125, '0.683', 80], [503.67578125, '10.390', 95]], [[491.31146240234375, '22.5%', 21], [490.5995788574219, '5.66', 36], [490.5077819824219, '0.577', 51], [490.5077819824219, '9.815', 66], [490.5077819824219, '0.682', 81], [490.5077819824219, '10.388', 96]], [[478.2154541015625, '22.4%', 22], [477.5035705566406, '5.64', 37], [477.4117736816406, '0.576', 52], [477.4117736816406, '9.807', 67], [477.4117736816406, '0.680', 82], [477.4117736816406, '10.370', 97]], [[465.11944580078125, '22.3%', 23], [464.4075622558594, '5.62', 38], [464.3157653808594, '0.574', 53], [464.3157653808594, '9.792', 68], [464.3157653808594,

In [55]:
lines = []
line = []

prev_block_no = raw_lines[0][0][2]

w_string = ""

for raw_line in raw_lines:
    
    prev_block_no = raw_line[0][2]

    for raw_word in raw_line:
        
        curr_block_no = raw_word[2]
        raw_word_string = raw_word[1]

        if curr_block_no == prev_block_no:
            
            if not w_string:
                w_string = w_string + raw_word_string
            else:
                w_string = w_string + " " + raw_word_string
        else:

            line.append(w_string)
            w_string = raw_word_string

        prev_block_no = curr_block_no

    line.append(w_string)
    w_string = ""

    lines.append(line)

    line = []

table_df = pd.DataFrame(lines)
table_df.head()


    



,0,1,2,3,4,5
0,EFF Code(%),Pmpp(W),Vmpp(V),Impp(A),Voc(V),Isc(A)
1,22.6%,5.69,0.579,9.835,0.683,10.390
2,22.5%,5.66,0.577,9.815,0.682,10.388
3,22.4%,5.64,0.576,9.807,0.680,10.370
4,22.3%,5.62,0.574,9.792,0.679,10.344


# TABULA TESTING

In [60]:
import tabula

In [87]:
table_area = [table_x0, 824.0 - table_y0, table_x1, 824.0 - table_y1]
table_area_alt = [824.0 - table_y0, table_x0, 824.0 - table_y1, table_x1]

print(table_area_alt)

dfs = tabula.read_pdf(
    input_path="../misc/1-4.pdf",
    output_format="dataframe",
    area=table_area_alt,
    pages=1,
    stream=True
)

[289.0, 251.0, 498.0, 573.0]


In [89]:
print(type(dfs))

for df in dfs:
    print(df.T.reset_index().T)
    print(df)

    print()

<class 'list'>
                 0        1        2        3       4       5
index  EFF Code(%)  Pmpp(W)  Vmpp(V)  Impp(A)  Voc(V)  Isc(A)
0            22.6%     5.69    0.579    9.835   0.683   10.39
1            22.5%     5.66    0.577    9.815   0.682  10.388
2            22.4%     5.64    0.576    9.807    0.68   10.37
3            22.3%     5.62    0.574    9.792   0.679  10.344
4            22.2%     5.59    0.571    9.792   0.678  10.334
5            22.1%     5.57    0.569    9.792   0.677  10.333
6            22.0%     5.54    0.568    9.759   0.677  10.313
7            21.9%     5.52    0.567    9.728   0.676   10.29
8            21.8%     5.49    0.567    9.693   0.675  10.254
9            21.7%     5.47    0.564    9.693   0.674  10.226
10           21.6%     5.44    0.563    9.658   0.671  10.221
11           21.5%     5.42    0.561    9.654   0.669  10.209
12           21.4%     5.39    0.558    9.663   0.667  10.195
13           21.3%     5.37    0.557     9.64   0.665  